In [21]:
import pandas as pd
import numpy as np
import networkx as nx
from typing import List
from collections import defaultdict

In [9]:
# fname = "../data/huang_emt_scc.gxml"
# fname = "../data/omnipath.gxml"
fname = "../data/tcell_scc.gxml"
g = nx.read_graphml(fname)

# Positive-cycle
A cycle is positive if the XOR of their "represses" of all the edges results in a positive

In [19]:
cycles = sorted(nx.simple_cycles(g))
cycle = cycles[0]
positive = True
print(positive, end ="")
for i, node in enumerate(cycle):
    next_node = cycle[(i + 1) % len(cycle)]
    repress = g.edges[node, next_node]['repress']
    positive ^= repress
    print(f" -> ({node}, {next_node}: {repress}) -> {positive}", end="")
    if (i + 1) % 3 == 0:
        print(f'\n{positive}', end="")

True -> (0, 1: False) -> True -> (1, 0: True) -> False

## Type 1 Motif

## Find all positive cycles

In [11]:
def is_positive(graph: nx.Graph, cycle: List[str]):
    positive = True
    for i, node in enumerate(cycle):
        next_node = cycle[(i + 1) % len(cycle)]
        repress = g.edges[node, next_node]['repress']
        positive ^= repress
    return positive

In [ ]:
all_cycles = list(nx.simple_cycles(g))
cycles = [c for c in nx.simple_cycles(g) if is_positive(g, c)]
len(all_cycles) - len(cycles)

1424

## Cycle Set Map
For each edge, create a set that tracks all the cycles that the edge is in

In [54]:
edge_cycles_mp = defaultdict(set)
for cycle in cycles:
    cycle_set = tuple(cycle)
    for i, node in enumerate(cycle):
        next_node = cycle[(i + 1) % len(cycle)]
        edge_cycles_mp[node, next_node].add(cycle_set)
nx.set_edge_attributes(g, edge_cycles_mp, "cycles")

## Type 1 Detection